<a href="https://colab.research.google.com/github/nicholasfurl/Great-Courses/blob/main/Lesson12_BayesRecommenderReviewers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#In this notebook, we will build a recomendation system for academic papers.

In [1]:
#we download the data we'll be using from GitHub. 
#vocab2.txt contains the words in the tiles of the academic papers, 
#cb.txt contains the titles of the papers.

!wget https://github.com/mlittmancs/great_courses_ml/raw/master/data/vocab2.txt
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/data/cb.txt

--2023-01-12 18:24:43--  https://github.com/mlittmancs/great_courses_ml/raw/master/data/vocab2.txt
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/data/vocab2.txt [following]
--2023-01-12 18:24:43--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/data/vocab2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134845 (132K) [text/plain]
Saving to: ‘vocab2.txt’

vocab2.txt          100%[===================>] 131.68K  --.-KB/s    in 0.02s   

2023-01-12 18:24:43 (7.75 MB/s) - ‘vocab2.txt’ saved [134845/134845]

--2023-01-12 18:24:43--  ht

In [7]:
#Read in data

# vocab2.txt contains the words in the tiles of the academic papers
vocab_dict = readvocab()

# cb.txt contains the titles of the papers.
(dat, labs) = getdat(vocab_dict)

In [3]:
#readvocab creates a vocab_dict with a count of the number of times a word occurs in our paper titles

#OK this looks like something included in a previous lesson that was then never used. 
#Probably he brings this out by habit for every bad of words model as a matter of course

# read in the vocabulary file 
def readvocab():

   # keep track of the number of words
    lexiconsize = 0

   # initialize an empty dictionary
    vocab_dict = {}

   # create a catch-all feature (vector component) for all unknown words
    vocab_dict["@unk"] = lexiconsize  #unk is unknown
    lexiconsize += 1

   # read in the vocabulary file
    with open("vocab2.txt", "r") as f:
        data = f.readlines()

   # Process the file a line at a time.
    for line in data:

        # The count is the first 3 characters
        count = int(line[0:4])

        # The word is the rest of the string
        token = line[5:-1]

       # Create a feature if it’s appeared at least twice
        if count > 1: 
            vocab_dict[token] = lexiconsize
            lexiconsize += 1
            
    # squirrel away the total size for later reference
    vocab_dict["@size"] = lexiconsize
    return(vocab_dict)

In [4]:
# Turn string str into a vector.
def tokenize(string, vocab_dict):

  #If I remember right, size just counted up the entries in the vocab dictionary
  #so this creates 0s corresponding to each of the elements in the vocab dictionary
  # initially the vector is all zeros
  vec = [0 for i in range(vocab_dict["@size"])]

  # for each word
  for t in string.split(" "): #Splits on tabe or space instead of comma? Then loops through each split-off string / word

   # if the word has a feature, add one to the corresponding feature
   #So vec will hold a frequency for each word in the dictionary. I remember this from other lesson ..
   #So a sparse representation of the title - frequencies of tiotle words over a large dictionary
    if t in vocab_dict: vec[vocab_dict[t]] += 1

   # otherwise, count it as an unk
    else: vec[vocab_dict["@unk"]] += 1
    
  return(vec)

In [6]:
# read in labeled examples and turn the strings into vectors
def getdat(vocab_dict):

    #List of titles
    with open("cb.txt", "r") as f:
        data = f.readlines()

    dat = []
    labs = []

    #
    for line in data:

        #Labs are changed to int and appended to lab list
        #I think these were machine learning versus robotics titles
        labs = labs + [int(line[0])]

        #tokenise the titles, which means create a vector corresponding to dictioonary entries that gives frequencies of each dictionary word in title (so rather sparse)
        dat = dat + [tokenize(line[2:], vocab_dict)]

    return(dat, labs)